<a href="https://colab.research.google.com/github/nguyenvuong1309/Do-an-lap-trinh-an-toan/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import pandas as pd
from numpy import zeros, newaxis
import numpy as np



In [ ]:
csv_file_path = '/content/drive/MyDrive/Colab/dataset/sample.csv'
df = pd.read_csv(csv_file_path)


In [ ]:
print(df.shape)

(181641, 52)


In [ ]:
print(df[:1])

   Index         1         2         3         4         5         6  \
0      0  0.252445 -0.119424 -0.035009 -0.059346 -0.057337  0.145114   

          7         8         9  ...        42        43        44        45  \
0 -0.249521  0.231396 -0.202908  ...  0.188895  0.113198 -0.120779  0.115096   

         46        47        48        49        50  label  
0 -0.233971 -0.074264 -0.472756 -0.001527 -0.665612    0.0  

[1 rows x 52 columns]


In [ ]:
dataset = df

In [ ]:
dataset = tf.constant(dataset)
dataset = dataset[1:,1:]

In [ ]:
def splits(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(DATASET_SIZE * TRAIN_RATIO))

  val_test_dataset = dataset.skip(int(DATASET_SIZE * TRAIN_RATIO))

  val_dataset = val_test_dataset.take(int(DATASET_SIZE * VAL_RATIO))

  test_dataset = val_test_dataset.skip(int(DATASET_SIZE * VAL_RATIO))

  return train_dataset, val_dataset, test_dataset

In [ ]:
len(dataset)

181640

In [ ]:
TRAIN_RATIO=0.6
VAL_RATIO=0.2
TEST_RATIO=0.2

#dataset = tf.data.Dataset.range(10)
#train_dataset, val_dataset, test_dataset = splits(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO)
train_dataset = dataset[:int(TRAIN_RATIO * len(dataset))]
val_dataset = dataset[int(TRAIN_RATIO * len(dataset)) : int(TRAIN_RATIO * len(dataset) + VAL_RATIO * len(dataset))]
test_dataset = dataset[(int(TRAIN_RATIO * len(dataset)) + int(VAL_RATIO * len(dataset))) :]

In [ ]:
print(train_dataset.shape)
print(val_dataset.shape)
print(test_dataset.shape)

(108984, 51)
(36328, 51)
(36328, 51)


In [ ]:
108981 + 36327 * 2

181635

In [ ]:
X_train = train_dataset[:,:-1]
y_train = train_dataset[:,-1]
print(X_train.shape)
print(y_train.shape)

(108984, 50)
(108984,)


In [ ]:
X_train = tf.expand_dims(X_train, axis=1)

In [ ]:
print(X_train.shape)

(108984, 1, 50)


In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=41)
y_train.shape

(108984, 41)

In [ ]:
X_test = test_dataset[:,:-1]
y_test = test_dataset[:,-1]
print(X_test.shape)
print(y_test.shape)
X_test = tf.expand_dims(X_test, axis=1)
print(X_test.shape)

(36328, 50)
(36328,)
(36328, 1, 50)


In [ ]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes=41)
y_test.shape

(36328, 41)

In [ ]:
model = tf.keras.models.Sequential([
     tf.keras.layers.InputLayer(input_shape=(1,50)),

    # tf.keras.layers.Conv2D(32, (3, 3), padding='same'),
    # tf.keras.layers.Activation('relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(41),
    tf.keras.layers.Activation('softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 1000)              51000     
                                                                 
 activation_2 (Activation)   (None, 1000)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               512512    
                                                                 
 activation_3 (Activation)   (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 41)                21033     
                                                                 
 activation_4 (Activation)   (None, 41)               

In [ ]:
 opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
 model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'],
                #run_eagerly=True
               )

In [ ]:
model.fit(X_train,y_train,epochs=3)

Epoch 1/3
3406/3406 [==============================] - 46s 13ms/step - loss: 1.0246 - accuracy: 0.7622
Epoch 2/3
3406/3406 [==============================] - 43s 13ms/step - loss: 0.8819 - accuracy: 0.7797
Epoch 3/3
3406/3406 [==============================] - 44s 13ms/step - loss: 0.8053 - accuracy: 0.7925


In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

1136/1136 - 3s - loss: 0.7626 - accuracy: 0.7944 - 3s/epoch - 3ms/step

Test accuracy: 0.7944009900093079


In [ ]:
#test